<a href="https://colab.research.google.com/github/utkar22/CSE508_Winter2023_A2_48/blob/main/IR_A2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pickle 
import numpy as np
import pandas as pd
from multiset import *
import os

# **Jaccard Coefficient**


In [41]:
mod_path = '/content/drive/MyDrive/IR_Assignments/A2'

In [ ]:
ds_path = '/content/drive/MyDrive/IR_Assignments/A2/Processed_Dataset'
doc_id_to_set = {}
idx = 1
for doc in sorted(os.listdir(ds_path)):
  mset = set()

  doc_id = int(doc[-4:])
  doc_path = ds_path + '/' + doc

  print(doc_id)

  with open(doc_path, 'r') as f:
    text = str(f.readlines()[0])

  tokens = text.split(' ')


  for token in tokens:
    mset.add(token)

  print(len(mset))
  print(len(tokens))
  doc_id_to_set[doc_id] = mset

print(len(doc_id_to_set))  
with open(mod_path+'/doc_id_to_mset.pkl', 'wb') as f:
  pickle.dump(doc_id_to_set, f)



In [60]:
#calculating Jaccard coefficient of each query document wise
def calc_jacc(query_tokens, doc_id_to_set):
  res = []
  res_set = {}
  query_set = set()
  for token in query_tokens: #make the set of the query tokens
    query_set.add(token)

  for doc_id, mset in doc_id_to_set.items(): #calculate the intersection and union of each document set and query set
    un, inter = len(query_set.union(mset)), len(query_set.intersection(mset))
    if un == 0: #edge case when the doc or query do not have any words in them
      res_set[doc_id] = 0
      continue
    
    jacc_coeff = inter/un #calculating the Jaccard Coefficient
    res_set[doc_id] = jacc_coeff
  res_set = {k:v for k,v in sorted(res_set.items(), key = lambda item:item[1], reverse = True)} #sorting in descending order on the basis of the Jaccard Coefficient
  idx = 0
  for doc_id, jacc_coeff in res_set.items(): # make the list of the top 10 documents on the basis of the Jaccard coefficient
    res.append((doc_id, jacc_coeff))
    if idx == 9:
      break
    idx+=1

  return res




In [65]:
#query processing and result printing

q = int(input('Enter number of queries: '))
while q>0:
  query = input('Enter query: ')

  query_tokens = query.split(' ')
  res = calc_jacc(query_tokens, doc_id_to_set)

  if res[0][1] == 0.0:
    print('No right document exists')
  else:
    print(res)
  q-=1

Enter number of queries: 3
Enter query: velocity pressure
[(194, 0.07142857142857142), (3, 0.058823529411764705), (215, 0.05714285714285714), (900, 0.05714285714285714), (241, 0.05555555555555555), (1139, 0.05405405405405406), (1283, 0.05263157894736842), (74, 0.046511627906976744), (770, 0.045454545454545456), (984, 0.045454545454545456)]
Enter query: helloo
No right document exists
Enter query: train
No right document exists
